In [ ]:
import subprocess
subprocess.check_output(["maturin", "develop"])
import time

In [ ]:
import numpy as np
import audioflux as af
from audioflux.type import SpectralFilterBankScaleType
from IPython.display import Audio, display, Image, HTML
import matplotlib.pyplot as plt
from audioflux.display import fill_spec
import random
import PIL
from tqdm.notebook import trange, tqdm
from copy import deepcopy
from IPython.display import clear_output
import librosa
from scipy.optimize import differential_evolution
import soundfile as sf
from multiprocessing import Pool
import os
from luthier.luthier import DspGraph, SineOscillator, Multiplier, HarmonicMultiplier, Sum, ADSR
from luthier.dsp import Sample
plt.rcParams['figure.dpi'] = 110
SAMPLE_RATE = 22050

In [ ]:
fm_sample_data, fm_sample_sample_rate = librosa.load('./fm_sample.wav')
print("Target sample rate", fm_sample_sample_rate)
print("Generator sample rate", SAMPLE_RATE)

fm_sample = Sample(fm_sample_data[100:])

fig, (ax, ax2) = plt.subplots(2, 1, sharex=False, sharey=False)
fm_sample.plot_spectrogram(ax)
ax2.plot(fm_sample.buffer[:400])

fm_sample.show_player()

In [ ]:
def one_operator_fm():
    graph = DspGraph()
    modulator = graph.add_sine(SineOscillator())
    _carrier = SineOscillator()
    carrier = graph.add_sine(_carrier)
    
    mod_index_scaler = graph.add_multiplier(Multiplier())
    mod_freq_mult = graph.add_harmonic_multiplier(HarmonicMultiplier())
    
    graph.patch(carrier, "output_output", 2, "input_input")
    graph.patch(graph.base_frequency_node_id, "output_output", carrier, "input_frequency")
    graph.patch(graph.base_frequency_node_id, "output_output", mod_freq_mult, "input_input" )
    graph.patch(mod_freq_mult, "output_output", modulator, "input_frequency")
    graph.patch(modulator, "output_output", mod_index_scaler, "input_input")
    graph.patch(mod_index_scaler, "output_output", carrier, "input_modulation")
    
    return graph

target_graph = one_operator_fm()
# inputs = target_graph.get_inputs()
# print(inputs)
# inputs[2] = 0.25
# target_graph.set_inputs(inputs)
target_graph.set_input(5, "input_scale", 0.75)
target_graph.set_input(6, "input_scale", 0.7)
# target_graph.set_input(6, "input_scale", -1.0)

target_graph.set_input(target_graph.amp_adsr_node_id , "input_attack", 0.001)
target_graph.set_input(target_graph.amp_adsr_node_id , "input_sustain", 0.02)
target_graph.set_input(target_graph.amp_adsr_node_id , "input_release", 0.3)
display(Image(target_graph.draw()))
target_audio = Sample(target_graph.play(num_samples = SAMPLE_RATE))


fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=False, sharey=False)
target_audio.plot_spectrogram(ax1)
target_audio.plot_waveform(ax2)
target_audio.plot_mfcc(ax3)
target_audio.show_player()
target_graph.num_inputs()

In [ ]:
# target_audio = fm_sample

MAX_ITERATIONS = 30
POPULATION_SIZE = 20

def optimise_freq_and_mod(x):
    graph = one_operator_fm()
    graph.set_inputs(x)
    generated_audio = Sample(graph.play(len(target_audio)))
    dist = target_audio.mfcc_distance(generated_audio)
    
    return dist

def show_comparison(x):
    graph = one_operator_fm()
    graph.set_inputs(x)

    generated_audio = Sample(graph.play(num_samples = SAMPLE_RATE * 1))
    fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, sharex=False, sharey=False)

    generated_audio.show_player()
    target_audio.show_player()

    generated_audio.plot_spectrogram(ax1, "Trained spectrogram")
    generated_audio.plot_waveform(ax5, title="Trained waveform")
    generated_audio.plot_mfcc(ax3, title="Trained MFCC")

    target_audio.plot_spectrogram(ax2, title="Target spectrogram")
    target_audio.plot_waveform(ax6, title="Target waveform")
    target_audio.plot_mfcc(ax4, title="Target MFCC")

    fig.tight_layout()
    plt.savefig(f'{time.time()}.png')
    plt.show()

target_fun_values = []
def callback(x, convergence):
    clear_output()
    target_fun = optimise_freq_and_mod(x)
    target_fun_values.append(target_fun)
    show_comparison(x)
    plt.plot(target_fun_values)
    plt.scatter(MAX_ITERATIONS, 0)
    plt.title(f"Training progress (problem size {len(x)})")
    plt.xlabel("Iteration number")
    plt.ylabel("MFCC distance")
    plt.show()

    # return True

num_params = one_operator_fm().num_inputs()
print("Solving for num inputs", num_params)
solution = differential_evolution(
    optimise_freq_and_mod,
    [ (-1, 1) for i in range(num_params) ],
    workers=8,
    maxiter=MAX_ITERATIONS,
    popsize=POPULATION_SIZE,
    # polish=True,
    disp=False,
    callback=callback
)
print(solution)

In [ ]:
mod_strengths = [ a / 100 for a in range(-100, 101, 10) ]

def get_distance_variable_mod_strength(strength):
    graph = one_operator_fm()
    graph.set_input(5, "input_scale", strength)
    generated_audio = Sample(graph.play(SAMPLE_RATE))
    dist = target_audio.mfcc_distance(generated_audio)
    return dist

with Pool() as p:
    results = list(tqdm(p.imap(get_distance_variable_mod_strength, mod_strengths), total=len(mod_strengths)))

plt.plot( mod_strengths, results )

In [ ]:
mod_freqs = [ a / 100 for a in range(-100, 100, 5) ]
results = []

def get_distance_variable_mod_freq(freq):
    graph = one_operator_fm()
    graph.set_input(6, "input_scale", freq)
    graph.set_input(5, "input_scale", 1.0)
    generated_audio = Sample(graph.play(len(target_audio)))
    dist = target_audio.mfcc_distance(generated_audio)
    return dist

with Pool(1) as p:
    results = list(tqdm(p.imap(get_distance_variable_mod_freq, mod_freqs), total=len(mod_freqs)))

plt.plot( mod_freqs, results )

In [ ]:
attack_values = [ a / 100 for a in range(-100, 100, 1) ]
results = []

# from frechet_audio_distance import FrechetAudioDistance

# # to use `vggish`
# frechet = FrechetAudioDistance(
#     model_name="vggish",
#     use_pca=False, 
#     use_activation=False,
#     verbose=False
# )

def get_distance_variable_attack(attack):
    graph = one_operator_fm()
    graph.set_input(graph.amp_adsr_node_id, "input_attack", attack)
    generated_audio = Sample(graph.play(SAMPLE_RATE))
    # dist = frechet.calculate_frechet_distance(target_audio.buffer, generated_audio.buffer, 0.1, 0.1)
    dist = target_audio.mfcc_distance(generated_audio)
    return dist

with Pool() as p:
    results = list(tqdm(p.imap(get_distance_variable_attack, attack_values), total=len(attack_values)))

plt.plot(attack_values, results)

In [ ]:
# from frechet_audio_distance import FrechetAudioDistance
# import numpy

# # to use `vggish`
# frechet = FrechetAudioDistance(
#     model_name="vggish",
#     use_pca=False, 
#     use_activation=False,
#     verbose=False
# )

# embeddings = frechet.model.forward(target_audio.buffer, SAMPLE_RATE)
# mu = np.mean(embeddings.detach().numpy())
# sigma = np.cov(embeddings.detach().numpy(), rowvar=False)

In [ ]:
STEP_VALUE = 20 # Lower means better resolution
mod_freqs = [ a / 100 for a in range(-100, 101, STEP_VALUE) ] 
mod_strengths = [ a / 100 for a in range(-100, 101, STEP_VALUE) ] 

test_cases = []
for freq_index, freq in enumerate(mod_freqs):
    for mod_index, mod_strength in enumerate(mod_strengths):    
        test_cases.append( ( (freq_index, freq), (mod_index, mod_strength) ) )

def get_distance_variable_mod_strength(test_case):
    (freq_index, freq), (mod_index, mod_strength) = test_case
    graph = one_operator_fm()

    graph.set_input(6, "input_scale", freq)
    graph.set_input(5, "input_scale", mod_strength)
    generated_audio = Sample(graph.play(SAMPLE_RATE * 1))

    dist = target_audio.mfcc_distance(generated_audio)
    
    return freq_index, mod_index, dist
    
with Pool() as p:
    pool_results = list(tqdm(p.imap(get_distance_variable_mod_strength, test_cases), total=len(test_cases)))  
    
results = np.zeros((len(mod_freqs), len(mod_strengths)))

for pool_result in pool_results:
    freq_index, mod_index, dist = pool_result
    results[freq_index][mod_index] = dist
    
fig, ax = plt.subplots()
im = ax.imshow(results)

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(mod_strengths)), labels=mod_strengths)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
ax.set_yticks(np.arange(len(mod_freqs)), labels=mod_freqs)
ax.set_ylabel("Modulation frequency")
ax.set_xlabel("Modulation strength")
fig.tight_layout()

In [ ]:
def multiplier_with_param(graph: DspGraph):
    mult = graph.add_multiplier(Multiplier())
    return mult

def adsr_with_params(graph: DspGraph):
    adsr = graph.add_adsr(ADSR())
    return adsr

def oscillator_with_multiplier(graph: DspGraph):
    osc = graph.add_sine(SineOscillator())
    mod_osc = graph.add_sine(SineOscillator())
    
    mod_osc_freq_mult = graph.add_harmonic_multiplier(HarmonicMultiplier())
    graph.set_input(mod_osc_freq_mult, "input_scale", random.uniform(-1.0, 1.0))
    
    graph.patch(graph.base_frequency_node_id, "output_output", mod_osc_freq_mult, "input_input")
    graph.patch(graph.base_frequency_node_id, "output_output", osc, "input_frequency")
    graph.patch(mod_osc_freq_mult, "output_output", mod_osc, "input_frequency")
    
    mod_adsr = adsr_with_params(graph)
    osc_adsr = adsr_with_params(graph)
    
    graph.patch(mod_osc, "output_output", mod_adsr, "input_input")
    graph.patch(osc, "output_output", osc_adsr, "input_input")
    
    graph.patch(mod_adsr, "output_output", osc, "input_modulation")
    mult = multiplier_with_param(graph)
    graph.patch(osc, "output_output", mult, "input_input")
    return mult


def oscillator_with_adsr(graph: DspGraph):
    mult = oscillator_with_multiplier(graph)
    adsr = adsr_with_params(graph)  
    graph.patch(mult, "output_output", adsr, "input_input")
    
    return adsr
    
def complex_fm():
    graph = DspGraph()
    mixer = graph.add_sum(Sum())
    graph.patch(mixer, "output_output", graph.amp_adsr_node_id, "input_input")

    osc_output_node = oscillator_with_adsr(graph)
    graph.patch(osc_output_node, "output_output", mixer, "input_in_1")

#     osc_output_node = oscillator_with_adsr(graph)
#     graph.patch(osc_output_node, "output_output", mixer, "input_in_2")

#     osc_output_node = oscillator_with_adsr(graph)
#     graph.patch(osc_output_node, "output_output", mixer, "input_in_3")
    
    return graph

graph = complex_fm()
    
# osc_output_node = oscillator_with_adsr(graph)
# graph.patch(osc_output_node, "output", mixer, "in_4")

# from copy import deepcopy
# best_graph = deepcopy(graph)

generated_audio = Sample(graph.play(SAMPLE_RATE * 2))
# generated_audio = graph.save_video(SAMPLE_RATE * 2)

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=False, sharey=False)
generated_audio.plot_spectrogram(ax1)
generated_audio.plot_waveform(ax2, num_samples=400)
generated_audio.plot_mfcc(ax3)
generated_audio.show_player()
plt.show()

display(Image(graph.draw()))
distances = []